In [170]:
from pydub import AudioSegment
from pydub.generators import Sine
import math
import musicpy as mp
from midi2audio import FluidSynth

color_to_freq_map = {
    "#00290e": 261.63,  # C4
    "#fefefe": 293.66,  # D4
    '#d11033': 329.63,  # E4
}

def hex_to_rgb(hex_color):
    hex_color = hex_color.lstrip('#')
    return tuple(int(hex_color[i:i+2], 16) for i in (0, 2, 4))

def rgb_to_hex(rgb):
    """Convert an RGB color to hexadecimal format."""
    return '#{:02x}{:02x}{:02x}'.format(*rgb)

def color_distance(rgb1, rgb2):
    """Calculate the Euclidean distance between two RGB colors."""
    return math.sqrt(sum((c1 - c2) ** 2 for c1, c2 in zip(rgb1, rgb2)))

def closest_color(rgb, color_map):
    """Find the closest color in the color_map to the given RGB color."""
    closest_colors = sorted(color_map.keys(), key=lambda color: color_distance(rgb, hex_to_rgb(color)))
    return closest_colors[0]

def freq_to_note_name(freq):
    # This is a simplification, in reality, you would need to account for tuning and the exact frequencies of notes
    notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    return notes[round(math.log2(freq / 440.0) * 12) % 12]

def colors_to_scale(main_colors, color_to_freq_map):

    notes = [freq_to_note_name(color_to_freq_map[closest_color(hex_to_rgb(color), color_to_freq_map)]) for color in main_colors]
    chord_obj = mp.chord(notes)
    scales = mp.alg.detect(chord_obj)
    return scales

def wich_instrument():
    instrument = 0
    for color in color_to_freq_map.values():
        instrument += color
    instrument = instrument % 128
    return int(instrument)

def divide_by_100(array):
    divided_array = []
    for sublist in array:
        divided_sublist = [value / 100 for value in sublist]
        divided_array.append(divided_sublist)
    return divided_array

def round_to_nearest(value):
    multiples = [6.25, 20, 12.5, 33, 50, 100]
    nearest = min(multiples, key=lambda x: abs(x - value))
    return nearest

def round_values(array):
    rounded_array = []
    for sublist in array:
        rounded_sublist = [round_to_nearest(value) for value in sublist]
        rounded_array.append(rounded_sublist)
    return rounded_array

def calculate_percentages(array):
    percentages_array = []
    for sublist in array:
        sublist_length = len(sublist)
        counts = {num: sublist.count(num) for num in set(sublist)}
        total_counts = sum(counts.values())
        percentages = [(count / total_counts) * 100 for count in counts.values()]
        rounded_percentages = [round(percentage) for percentage in percentages]
        diff = 100 - sum(rounded_percentages)
        # Distribuer la différence d'arrondi en ajoutant ou en soustrayant 1 au premier ou au dernier élément
        if diff > 0:
            rounded_percentages[-1] += diff
        elif diff < 0:
            rounded_percentages[0] += diff
        percentages_array.append(rounded_percentages)
    percentages_array = round_values(percentages_array)
    for sublist in percentages_array:
        distinct_values = set(sublist)
        if len(distinct_values) > 1 and 100 in sublist:
            sublist.remove(100)
    return divide_by_100(percentages_array)


In [175]:
def compress_list(arr):
    compressed_lists = []
    for lst in arr:
        compressed_list = [lst[0]]
        for i in range(1, len(lst)):
            if lst[i] != lst[i - 1]:
                compressed_list.append(lst[i])
        compressed_lists.append(compressed_list)
    return compressed_lists


In [140]:
def image_to_music(main_colors, pixels, note_duration=1000, min_length=0.125, outputfile="output_song.wav"):

    # Define the minimum length as a percentage of the note_duration
    min_note_length = note_duration * min_length

    song = AudioSegment.silent(duration=0)  # Start with a silent segment

    for row in pixels:
        row_length = len(row)
        current_color = main_colors[row[0]]
        current_count = 1

        note_groups = []

        # Iterate over the row to group colors
        for idx in row[1:]:
            hex_color = main_colors[idx]
            if hex_color == current_color:
                current_count += 1
            else:
                # Process the previous group
                note_groups.append((current_color, current_count))
                current_color = hex_color
                current_count = 1
        # Add the last color group
        note_groups.append((current_color, current_count))

        # Calculate the total length for notes that are above the minimum length
        total_above_min = sum(count for color, count in note_groups if count / row_length >= min_length)

        # Generate notes
        for hex_color, count in note_groups:
            proportion = count / row_length
            if proportion < min_length:
                note_time = min_note_length
            else:
                note_time = note_duration * (count / total_above_min)

            rgb_color = hex_to_rgb(hex_color)
            closest_hex = closest_color(rgb_color, color_to_freq_map)
            freq = color_to_freq_map[closest_hex]
            note = Sine(freq).to_audio_segment(duration=int(note_time))
            song += note

    outputfile
    song.export(outputfile, format="wav")

    print("Generated music saved to:", outputfile)
    return outputfile  # To allow accessing the output location


In [141]:
from PIL import Image
import numpy as np
from sklearn.cluster import KMeans

def load_and_cluster(path, n_clusters, display=False):
    image = Image.open(path)
    x, y = image.size

    h = 150 / float(y)

    redim = min(x, h)

    new_h = int(y * redim)

    image = image.resize((x, new_h), Image.LANCZOS)

    x, y = image.size

    if image.mode != 'RGB':
        image = image.convert('RGB')

    pixels = list(image.getdata())
    pixels_list = [pixels[i * x:(i + 1) * x] for i in range(y)]
    image_array = np.array(image)
    pixels = image_array.reshape(-1, 3)

    kmeans = KMeans(n_clusters=n_clusters)
    kmeans.fit(pixels)
    labels = kmeans.labels_

    for i in range(y):
        for j in range(x):
            pixel_index = i * x + j
            pixels_list[i][j] = labels[pixel_index]
    centers = [rgb_to_hex(i) for i in kmeans.cluster_centers_.astype(int)]
    return centers, pixels_list

In [176]:
main_colors, pixels_list = load_and_cluster("Flag_of_the_USVI.png", 6)
print(colors_to_scale(main_colors, color_to_freq_map))
print(main_colors)
print(wich_instrument())
result = calculate_percentages(pixels_list)
pixels_compressed = compress_list(pixels_list)
percentages_array = [result]
print(pixels_compressed)

F7sus2 omit C
['#fefefe', '#825c3c', '#f1c33d', '#18315b', '#007fc4', '#14100c']
116
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0, 2, 1, 2, 0, 2, 1, 0], [0, 2, 1, 3, 1, 2, 1, 2, 0, 1, 0, 1, 2, 1, 3, 1, 0], [0, 1, 5, 1, 2, 1, 2, 0, 1, 2, 1, 0, 2, 1, 2, 1, 5, 3, 0], [0, 5, 2, 1, 0, 1, 3, 5, 1, 2, 1, 2, 0, 2, 1, 2, 5, 3, 0], [0, 3, 5, 1, 2, 1, 2, 0, 1, 3, 5, 1, 2, 1, 5, 2, 1, 5, 3, 1, 0, 2, 1, 2, 1, 5, 1, 0], [0, 1, 3, 5, 1, 2, 1, 2, 0, 2, 1, 3, 5, 1, 2, 1, 5, 2, 1, 2, 1, 5, 3, 1, 0, 2, 1, 2, 1, 3, 1, 0], [0, 1, 2, 1, 2, 0, 1, 2, 1, 5, 1, 2, 1, 5, 1, 2, 1, 2, 1, 2, 1, 5, 1, 0, 1, 2, 1, 0], [0, 3, 5, 1, 2, 1, 2, 0, 1, 5, 1, 2, 1, 5, 1, 2, 1, 5, 1, 2, 1, 5, 3, 0, 2, 1, 2, 1, 5, 1, 0], [0, 5, 2, 1, 2, 1, 2, 0, 3, 5, 1, 2, 1, 2, 1, 2, 5, 2, 0, 2, 1, 2, 1, 2, 5, 0], [0, 1, 5, 2, 1, 2, 1, 3, 1, 2, 0, 5, 2, 1, 2, 1, 5, 1, 2, 1, 2, 5, 0, 2, 1, 3, 1, 2, 1, 2, 1, 5, 3, 0], [0, 2, 1, 5, 1, 2, 1, 2, 1, 2, 0, 1, 5, 1, 5, 1, 0, 1, 3, 5, 1, 2, 1, 5, 1, 2, 1, 5, 3, 

In [143]:
image_to_music(main_colors, pixels_list, min_length=0.125)

KeyboardInterrupt: 